In [ ]:
import pokemontcgsdk 
from pokemontcgsdk import Card, Set, RestClient
from PIL import Image
from IPython.display import display
from io import BytesIO
import requests
import pandas as pd
from IPython.display import display, HTML


api_key = "api-key"

RestClient.configure(api_key)

In [2]:
allSets = Set.all()

sets = pd.DataFrame(columns=["id", "name", "series", "release", "goCode"])

for s in allSets:
    sets.loc[len(sets)] = [s.id, s.name, s.series, s.releaseDate, s.ptcgoCode]


In [3]:
display(HTML(f"<div style='height: 250px; overflow:auto;'> {sets.to_html()} </div>"))

,id,name,series,release,goCode
0,base1,Base,Base,1999/01/09,BS
1,base2,Jungle,Base,1999/06/16,JU
2,basep,Wizards Black Star Promos,Base,1999/07/01,PR
3,base3,Fossil,Base,1999/10/10,FO
4,base4,Base Set 2,Base,2000/02/24,B2
5,base5,Team Rocket,Base,2000/04/24,TR
6,gym1,Gym Heroes,Gym,2000/08/14,G1
7,gym2,Gym Challenge,Gym,2000/10/16,G2
8,neo1,Neo Genesis,Neo,2000/12/16,N1
9,neo2,Neo Discovery,Neo,2001/06/01,N2


In [4]:
sv10 = Card.where(q="set.id:sv10")

cards = pd.DataFrame(columns=["id", "name", "supertype", "image"])

for card in sv10:
    image = card.images.small
    cards.loc[len(cards)] = [card.id, card.name, card.supertype, image]
    
     

In [5]:
print(cards)

           id                     name supertype  \
0      sv10-1           Ethan's Pinsir   Pokémon   
1      sv10-2                    Yanma   Pokémon   
2      sv10-3               Yanmega ex   Pokémon   
3      sv10-4                   Pineco   Pokémon   
4      sv10-5                Shroomish   Pokémon   
..        ...                      ...       ...   
239  sv10-240  Team Rocket's Mewtwo ex   Pokémon   
240  sv10-241    Cynthia's Garchomp ex   Pokémon   
241  sv10-242  Team Rocket's Crobat ex   Pokémon   
242  sv10-243            Jamming Tower   Trainer   
243  sv10-244                 Levincia   Trainer   

                                         image  
0      https://images.pokemontcg.io/sv10/1.png  
1      https://images.pokemontcg.io/sv10/2.png  
2      https://images.pokemontcg.io/sv10/3.png  
3      https://images.pokemontcg.io/sv10/4.png  
4      https://images.pokemontcg.io/sv10/5.png  
..                                         ...  
239  https://images.pokemontcg.i

In [6]:
import torch
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import ipywidgets

In [7]:
clipper = SentenceTransformer('clip-ViT-L-14')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
image_embeddings = []

for index, c in tqdm(cards.iterrows(), total=len(cards), desc="Processing Rows"):
    img = requests.get(c["image"]).content
    img = Image.open(BytesIO(img))

    emb = clipper.encode(img)
    image_embeddings.append(emb)


Processing Rows:   0%|          | 0/244 [00:00<?, ?it/s]

In [9]:
cards["emb"] = image_embeddings

In [10]:
cards.to_csv("SV10_truth.csv", index = False)
cards.to_pickle("SV10_truth.pkl")